In [3]:
import random
import sisl
import torch
import torch.optim as optim
import numpy as np
import yaml

from e3nn import o3
from pathlib import Path
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch_geometric.loader import DataLoader
from mace.modules import MACE, RealAgnosticResidualInteractionBlock
from graph2mat.bindings.e3nn import E3nnGraph2Mat
from graph2mat.models import MatrixMACE
from graph2mat.bindings.torch import TorchBasisMatrixDataset
from graph2mat import (
    PointBasis,
    BasisTableWithEdges,
    BasisConfiguration,
    MatrixDataProcessor,
)

def load_config(path="../config.yaml"):
    with open(path, "r") as f:
        return yaml.safe_load(f)
    
def flatten(xss):
    return [x for xs in xss for x in xs]

/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.

In [4]:
def get_basis_from_structures_paths(paths, verbose=False, num_unique_z=None):
    """_summary_

    Args:
        paths (list[str]): _description_
        verbose (bool, optional): _description_. Defaults to False.
        num_unique_z (int, optional): If known, input the total number of different atom types here for faster performance.

    Returns:
        list[PointBasis]: _description_
    """
    if verbose:
        print("="*60)
        print("Basis computation.")
        print(f"Number of structures to look on: {len(paths)}")
        print("Looking for unique atoms in each structure...")

    unique_atom_types = []
    unique_atom_types_path_idx = []

    # Look for all atom types in your list of structures
    iterator = tqdm(enumerate(paths)) if verbose else enumerate(paths)
    for i, path in iterator:
        if num_unique_z is not None and len(unique_atom_types) == num_unique_z:
            print("Found enough basis points. Breaking the search...")
            break
        geometry = sisl.get_sile(path / "aiida.fdf").read_geometry()
        for z in geometry.atoms.Z:
            if z not in unique_atom_types:
                unique_atom_types.append(z)
                unique_atom_types_path_idx.append(i)
            if num_unique_z is not None and len(unique_atom_types) == num_unique_z:
                print("Found enough basis points. Breaking the search...")
                break
        # print("n_atoms= ", len(geometry.atoms.Z))

    if verbose:
        print(f"Found the following atomic numbers: {unique_atom_types}")
        print(f"Corresponding path indices: {unique_atom_types_path_idx}")

    # Build the basis
    basis = []
    unique_atom_types_basis = []
    for path_idx in unique_atom_types_path_idx:
        geometry = sisl.get_sile(paths[path_idx] / "aiida.fdf").read_geometry()
        for atom in geometry.atoms:
            if atom.Z not in unique_atom_types_basis:
                basis.append(PointBasis.from_sisl_atom(atom))
                unique_atom_types_basis.append(atom.Z)

    basis.sort(key=lambda x: x.type)
    unique_atom_types_basis.sort()

    if verbose:
        print(f"Basis with {len(basis)} elements built!")
        [print(f"\nBasis for atom {i}.\n\tAtom type: {basis[i].type}\n\tBasis: {basis[i].basis}\n\tBasis convention: {basis[i].basis_convention}\n\tR: {basis[i].R}") for i in range(len(basis))]

    return basis

In [5]:
# === Configuration load ===
directory = Path("../results/hamiltonian") # * Write here the directory where the model is stored
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device=torch.device("cpu")
filename = "model_epoch_4500.tar"
print(f"Loading model {directory / filename}...")


config = load_config(directory / "config.yaml")

# === List of paths to all structures ===
parent_path = Path('../dataset')
n_atoms_paths = list(parent_path.glob('*/'))
paths = []
for n_atoms_path in n_atoms_paths:
    structure_paths = list(n_atoms_path.glob('*/'))
    paths.append(structure_paths)
paths = flatten(paths)

random.seed(config["dataset"]["seed"])
random.shuffle(paths)



# == Basis creation === 
basis = get_basis_from_structures_paths(paths, verbose=False, num_unique_z=config["dataset"].get("num_unique_z", None))
table = BasisTableWithEdges(basis)



# === Dataset creation ===
processor = MatrixDataProcessor(basis_table=table, symmetric_matrix=True, sub_point_matrix=False)
embeddings_configs = []
for i, path in enumerate(paths):

    # We need to keep track of the training/val splits, so we can't plot more than used for training (at least for training dataset)
    if i==20:
        break
    
    print(f"Processing structure {i+1} of {len(paths)}...")

    # Load the structure config
    file = sisl.get_sile(path / "aiida.fdf")
    file_h = sisl.get_sile(path / "aiida.HSX")
    geometry = file.read_geometry()

    # Load the true hamiltonian
    true_h = file_h.read_hamiltonian()

    embeddings_config = BasisConfiguration.from_matrix(
        matrix = true_h,
        geometry = geometry,
        labels = True,
        metadata={
            "atom_types": torch.from_numpy(geometry.atoms.Z), # Unlike point_types, this is not rescaled.,
            "path": path
        },
    )

    embeddings_configs.append(embeddings_config)

dataset = TorchBasisMatrixDataset(embeddings_configs, data_processor=processor)

# Split and stratify
# n_atoms_list = [dataset[i].num_nodes for i in range(len(dataset))] if config["dataset"]["stratify"] == True else None
n_atoms_list = None
train_dataset, val_dataset = train_test_split(
    dataset, 
    train_size=config["dataset"]["train_split_ratio"],
    stratify=n_atoms_list,
    random_state=None # Dataset already shuffled (paths)
)

Loading model ../results/hamiltonian/model_epoch_4500.tar...
Found enough basis points. Breaking the search...
Found enough basis points. Breaking the search...
Processing structure 1 of 1311...
Processing structure 2 of 1311...
Processing structure 3 of 1311...


/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(
/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(
/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(
/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(
/home/alapena/minico

Processing structure 4 of 1311...
Processing structure 5 of 1311...
Processing structure 6 of 1311...
Processing structure 7 of 1311...


/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(
/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(


Processing structure 8 of 1311...
Processing structure 9 of 1311...
Processing structure 10 of 1311...


...

/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(
/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(
/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(


Processing structure 13 of 1311...
Processing structure 14 of 1311...
Processing structure 15 of 1311...
Processing structure 16 of 1311...
Processing structure 17 of 1311...


/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(
/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(
/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(
/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(


Processing structure 18 of 1311...
Processing structure 19 of 1311...
Processing structure 20 of 1311...


/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(
/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(
/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/graph2mat/core/data/matrices/__init__.py:40: UserWarning: <class 'sisl.physics.Hamiltonian'> is not a known matrix type key, falling back to generic OrbitalMatrix class.
  warn(


In [40]:
Path(dataset[0].metadata["path"])

PosixPath('../dataset/SHARE_OUTPUTS_8_ATOMS/907c-7487-46bd-8ad3-65b031f7f12b')

In [6]:
import numpy as np

kxs = np.linspace(0,1, num=40)
kys = np.linspace(0,1, num=40)
kzs = np.linspace(0,1, num=40) # * Change the resolution here
k_dir_x = geometry.rcell[:,0]
k_dir_y = geometry.rcell[:,1]
k_dir_z = geometry.rcell[:,2]
k_path_x=np.array([kx*k_dir_x for kx in kxs])
k_path_y=np.array([ky*k_dir_y for ky in kys])
k_path_z=np.array([kz*k_dir_z for kz in kzs])
k_path=np.concatenate([k_path_x, k_path_y, k_path_z])

In [7]:
k_path

array([[0.        , 0.        , 0.        ],
       [0.04475216, 0.        , 0.        ],
       [0.08950431, 0.        , 0.        ],
       [0.13425647, 0.        , 0.        ],
       [0.17900863, 0.        , 0.        ],
       [0.22376078, 0.        , 0.        ],
       [0.26851294, 0.        , 0.        ],
       [0.31326509, 0.        , 0.        ],
       [0.35801725, 0.        , 0.        ],
       [0.40276941, 0.        , 0.        ],
       [0.44752156, 0.        , 0.        ],
       [0.49227372, 0.        , 0.        ],
       [0.53702588, 0.        , 0.        ],
       [0.58177803, 0.        , 0.        ],
       [0.62653019, 0.        , 0.        ],
       [0.67128235, 0.        , 0.        ],
       [0.7160345 , 0.        , 0.        ],
       [0.76078666, 0.        , 0.        ],
       [0.80553882, 0.        , 0.        ],
       [0.85029097, 0.        , 0.        ],
       [0.89504313, 0.        , 0.        ],
       [0.93979528, 0.        , 0.        ],
       [0.

In [ ]:
import plotly.graph_objects as go
import numpy as np

def plot_predictions_vs_truths(predictions, truths, series_names=None, 
                               title='True vs Predicted Values', 
                               xaxis_title='True Values', 
                               yaxis_title='Predicted Values', 
                               legend_title='Series',
                               show_diagonal=True, 
                               show_points=True,
                               path=None):
    """
    Plots true values vs predictions for multiple series using Plotly.
    
    Parameters:
    predictions (np.ndarray): 2D array of predicted values (rows: series, columns: points).
    truths (np.ndarray): 2D array of true values (same shape as predictions).
    series_names (list, optional): Names for each series. Defaults to generic names.
    title (str, optional): Plot title.
    xaxis_title (str, optional): X-axis title.
    yaxis_title (str, optional): Y-axis title.
    legend_title (str, optional): Legend title.
    show_diagonal (bool, optional): Whether to show the diagonal line.
    show_points (bool, optional): Whether points are visible by default (False sets to 'legendonly').
    path (str, optional): Path to save the plot as HTML.
    """
    # Validate input shapes
    if predictions.shape != truths.shape:
        raise ValueError("predictions and truths must have the same shape")
    
    n_series = predictions.shape[0]
    
    # Generate default series names if not provided
    if series_names is None:
        series_names = [f'Series {i+1}' for i in range(n_series)]
    elif len(series_names) != n_series:
        raise ValueError("series_names length must match number of series")
    
    # Create traces for each series
    traces = []
    for i in range(n_series):
        trace = go.Scatter(
            x=truths[i],
            y=predictions[i],
            mode='markers',
            name=series_names[i],
            visible=None if show_points else 'legendonly'
        )
        traces.append(trace)
    
    # Create diagonal line trace if enabled
    if show_diagonal:
        all_values = np.concatenate([truths.flatten(), predictions.flatten()])
        min_val, max_val = min(all_values), max(all_values)
        diagonal_trace = go.Scatter(
            x=[min_val, max_val],
            y=[min_val, max_val],
            mode='lines',
            line=dict(dash='dash', color='gray'),
            name='Perfect Prediction'
        )
        traces.append(diagonal_trace)
    
    # Create figure and update layout
    fig = go.Figure(data=traces)
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        legend_title=legend_title
    )
    
    # Save to HTML if path is provided
    if path:
        fig.write_html(path)
    
    return fig

# Example usage
np.random.seed(42)
# Generate random data: 3 series, 50 points each
truths = np.random.rand(3, 50) * 100
predictions = truths + np.random.randn(3, 50) * 10  # Add noise

# Generate plot
fig = plot_predictions_vs_truths(
    predictions=predictions,
    truths=truths,
    series_names=['Model A', 'Model B', 'Model C'],
    title='Model Predictions vs True Values',
    xaxis_title='Measured Values',
    yaxis_title='Predicted Values',
    legend_title='Models',
    show_diagonal=True,
    show_points=True,
    path=
)
fig

v

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Model A',
              'type': 'scatter',
              'x': {'bdata': ('AHi6Dx26QkByv9JRksRXQLfAz9/CTF' ... 'TeAEpADlJMPuRVS0DYM+QoRnwyQA=='),
                    'dtype': 'f8'},
              'y': {'bdata': ('u6hyOOTDQUBWixDWj4VYQLiTqrKLNl' ... 'ZW8E1ATPrca3KQUEBG4lWXp44kQA=='),
                    'dtype': 'f8'}},
             {'mode': 'markers',
              'name': 'Model B',
              'type': 'scatter',
              'x': {'bdata': ('7NNBdFc9WEClzSee2WBTQMQe3BDLfF' ... 'iGYEVApQPvT9ZVBEC6AAuDCpQlQA=='),
                    'dtype': 'f8'},
              'y': {'bdata': ('1vTH/+alWkC80BWlB2lUQB5sCGfpil' ... '4MDktAn1j+xPIpKEAyF8IGlU0xQA=='),
                    'dtype': 'f8'}},
             {'mode': 'markers',
              'name': 'Model C',
              'type': 'scatter',
              'x': {'bdata': ('RYyTfbIkCUCkl7iiDdJPQFd7iluDbz' ... '9KIklAogynk2yXFEBcOtB3Wd07QA=='),
                    'dtype': 'f8'},
              'y': {'bdata': ('AAqlqVMEhL/oBLxVxM5RQOzZ/p8UtT' ... 'oZ1khAKiLe0clDO8CKp4zI6p4xQA=='),
                    'dtype': 'f8'}},
             {'line': {'color': 'gray', 'dash': 'dash'},
              'mode': 'lines',
              'name': 'Perfect Prediction',
              'type': 'scatter',
              'x': [-27.264798275691795, 118.89554147429922],
              'y': [-27.264798275691795, 118.89554147429922]}],
    'layout': {'legend': {'title': {'text': 'Models'}},
               'template': '...',
               'title': {'text': 'Model Predictions vs True Values'},
               'xaxis': {'title': {'text': 'Measured Values'}},
               'yaxis': {'title': {'text': 'Predicted Values'}}}
})